In [38]:
import pandas as pd

In [39]:
df= pd.read_csv('partidos.csv',index_col='Unnamed: 0')
df.head()

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS
0,52023,1610612737,ATL,Atlanta Hawks,52300111,2024-04-17,ATL @ CHI,L,241,116,...,0.821,8.0,26.0,34.0,30,4.0,2,9,16,-15.0
1,22023,1610612737,ATL,Atlanta Hawks,22301188,2024-04-14,ATL @ IND,L,241,115,...,0.926,9.0,23.0,32.0,25,6.0,5,15,12,-42.0
2,22023,1610612737,ATL,Atlanta Hawks,22301178,2024-04-12,ATL @ MIN,L,240,106,...,0.680,9.0,31.0,40.0,23,4.0,1,14,25,-3.0
3,22023,1610612737,ATL,Atlanta Hawks,22301159,2024-04-10,ATL vs. CHA,L,240,114,...,0.625,7.0,31.0,38.0,35,7.0,2,16,20,-1.0
4,22023,1610612737,ATL,Atlanta Hawks,22301147,2024-04-09,ATL vs. MIA,L,292,111,...,0.786,17.0,42.0,59.0,28,13.0,2,15,23,-6.0


# DATA CLEANING

## ME QUEDO CON LAS FILAS IMPORTANTES
* 1º Convierto a temporal el tipo de dato de la columna 'GAME_DATE'
* 2º Me quedo con los posteriores al inicio de temporada 23/24. que según don google es el 25 de octubre.
  

In [40]:
df['GAME_DATE']= pd.to_datetime(df['GAME_DATE'])

In [41]:
df = df[df['GAME_DATE']>'2023-10-22'].sort_values(by='GAME_DATE')

Tengo que encontrar una forma de diferenciar quién juega de local y quien de visitante

* Descargo otro df en data/calendario/calendario.ipynb y lo limpio para hacer un merge con este df

In [42]:
calendario=pd.read_csv(r'calendario\calendario.csv')

In [43]:
calendario.head()

,Unnamed: 0,Date,Visitor/Neutral,Home/Neutral
0,0,2023-10-24,Los Angeles Lakers,Denver Nuggets
1,1,2023-10-24,Phoenix Suns,Golden State Warriors
2,2,2023-10-25,Houston Rockets,Orlando Magic
3,3,2023-10-25,Boston Celtics,New York Knicks
4,4,2023-10-25,Washington Wizards,Indiana Pacers


In [44]:
cal_eq_local=calendario[['Date','Home/Neutral']]

In [45]:
def coincidir_fechas_df(fecha):
    if (fecha.month >9) & (fecha.day>9):
        año=str(fecha.year)
        mes=str(fecha.month)
        dia =str(fecha.day)
    elif (fecha.month >9) & (fecha.day<9):
        año=str(fecha.year)
        mes=str(fecha.month)
        dia ='0'+str(fecha.day)
    elif (fecha.month <9) & (fecha.day>9):
        año=str(fecha.year)
        mes='0'+str(fecha.month)
        dia =str(fecha.day)
    else :
        año=str(fecha.year)
        mes='0'+str(fecha.month)
        dia ='0'+str(fecha.day)
    fecha_str= año+'-'+mes+'-'+dia
    return fecha_str
df['fecha'] = df['GAME_DATE'].apply(coincidir_fechas_df)
df['fecha']

93    2023-10-24
85    2023-10-24
82    2023-10-24
88    2023-10-24
88    2023-10-25
         ...    
1     2024-05-26
0     2024-05-27
0     2024-05-27
0     2024-05-28
0     2024-05-28
Name: fecha, Length: 2709, dtype: object

In [46]:
print(df['fecha'].iloc[0])
print(calendario['Date'][0])  
calendario['Date'][0] == df['fecha'].iloc[0]

2023-10-24
2023-10-24


True

In [47]:
df['merge_column']= df['fecha'] + df['TEAM_NAME']

In [48]:
calendario['merge_column']=calendario['Date']+calendario['Home/Neutral']
calendario

,Unnamed: 0,Date,Visitor/Neutral,Home/Neutral,merge_column
0,0,2023-10-24,Los Angeles Lakers,Denver Nuggets,2023-10-24Denver Nuggets
1,1,2023-10-24,Phoenix Suns,Golden State Warriors,2023-10-24Golden State Warriors
2,2,2023-10-25,Houston Rockets,Orlando Magic,2023-10-25Orlando Magic
3,3,2023-10-25,Boston Celtics,New York Knicks,2023-10-25New York Knicks
4,4,2023-10-25,Washington Wizards,Indiana Pacers,2023-10-25Indiana Pacers
...,...,...,...,...,...
1309,36,2024-05-25,Boston Celtics,Indiana Pacers,2024-05-25Indiana Pacers
1310,37,2024-05-26,Minnesota Timberwolves,Dallas Mavericks,2024-05-26Dallas Mavericks
1311,38,2024-05-27,Boston Celtics,Indiana Pacers,2024-05-27Indiana Pacers
1312,39,2024-05-28,Minnesota Timberwolves,Dallas Mavericks,2024-05-28Dallas Mavericks


In [49]:
df.merge(calendario,on='merge_column',how='outer')

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,...,BLK,TOV,PF,PLUS_MINUS,fecha,merge_column,Unnamed: 0,Date,Visitor/Neutral,Home/Neutral
0,22023.0,1.610613e+09,ATL,Atlanta Hawks,22300172.0,2023-11-09,ATL @ ORL,W,240.0,120.0,...,7.0,17.0,25.0,1.0,2023-011-09,2023-011-09Atlanta Hawks,NaN,NaN,NaN,NaN
1,22023.0,1.610613e+09,ATL,Atlanta Hawks,22300172.0,2023-11-09,ATL @ ORL,W,240.0,120.0,...,7.0,17.0,25.0,1.0,2023-011-09,2023-011-09Atlanta Hawks,NaN,NaN,NaN,NaN
2,22023.0,1.610613e+09,IND,Indiana Pacers,22300171.0,2023-11-09,IND vs. MIL,W,239.0,126.0,...,6.0,11.0,25.0,2.0,2023-011-09,2023-011-09Indiana Pacers,NaN,NaN,NaN,NaN
3,22023.0,1.610613e+09,MIL,Milwaukee Bucks,22300171.0,2023-11-09,MIL @ IND,L,240.0,124.0,...,5.0,14.0,19.0,-2.0,2023-011-09,2023-011-09Milwaukee Bucks,NaN,NaN,NaN,NaN
4,22023.0,1.610613e+09,ORL,Orlando Magic,22300172.0,2023-11-09,ORL vs. ATL,L,240.0,119.0,...,6.0,19.0,24.0,-1.0,2023-011-09,2023-011-09Orlando Magic,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2752,42023.0,1.610613e+09,BOS,Boston Celtics,42300304.0,2024-05-27,BOS @ IND,W,240.0,105.0,...,6.0,11.0,18.0,3.0,2024-05-27,2024-05-27Boston Celtics,NaN,NaN,NaN,NaN
2753,42023.0,1.610613e+09,IND,Indiana Pacers,42300304.0,2024-05-27,IND vs. BOS,L,240.0,102.0,...,4.0,14.0,19.0,-3.0,2024-05-27,2024-05-27Indiana Pacers,38.0,2024-05-27,Boston Celtics,Indiana Pacers
2754,42023.0,1.610613e+09,DAL,Dallas Mavericks,42300314.0,2024-05-28,DAL vs. MIN,L,239.0,100.0,...,4.0,13.0,22.0,-5.0,2024-05-28,2024-05-28Dallas Mavericks,39.0,2024-05-28,Minnesota Timberwolves,Dallas Mavericks
2755,42023.0,1.610613e+09,MIN,Minnesota Timberwolves,42300314.0,2024-05-28,MIN @ DAL,W,240.0,105.0,...,4.0,14.0,23.0,5.0,2024-05-28,2024-05-28Minnesota Timberwolves,NaN,NaN,NaN,NaN


In [50]:
df.drop_duplicates(inplace=True)

In [51]:
df['equipo_1']=df['MATCHUP'].str[:3]
df['equipo_2']=df['MATCHUP'].str[-3:]
df.head()

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,...,AST,STL,BLK,TOV,PF,PLUS_MINUS,fecha,merge_column,equipo_1,equipo_2
93,22023,1610612743,DEN,Denver Nuggets,22300061,2023-10-24,DEN vs. LAL,W,240,119,...,29,9.0,6,11,15,12.0,2023-10-24,2023-10-24Denver Nuggets,DEN,LAL
85,22023,1610612756,PHX,Phoenix Suns,22300062,2023-10-24,PHX @ GSW,W,241,108,...,23,5.0,7,19,22,4.0,2023-10-24,2023-10-24Phoenix Suns,PHX,GSW
82,22023,1610612744,GSW,Golden State Warriors,22300062,2023-10-24,GSW vs. PHX,L,240,104,...,19,11.0,6,11,23,-4.0,2023-10-24,2023-10-24Golden State Warriors,GSW,PHX
88,22023,1610612747,LAL,Los Angeles Lakers,22300061,2023-10-24,LAL @ DEN,L,239,107,...,23,5.0,4,11,18,-12.0,2023-10-24,2023-10-24Los Angeles Lakers,LAL,DEN
88,22023,1610612748,MIA,Miami Heat,22300068,2023-10-25,MIA vs. DET,W,241,103,...,22,11.0,3,7,18,1.0,2023-10-25,2023-10-25Miami Heat,MIA,DET


In [52]:
df_prueba_merge= df.merge(calendario,on='merge_column',how='outer')

In [53]:
df_visitantes = df_prueba_merge[df_prueba_merge['Date'].isna()]


In [54]:
df_locales = df_prueba_merge.dropna(axis=0,subset='Visitor/Neutral')

In [55]:
df_locales['MATCHUP'] = df_locales['MATCHUP'].astype(str)
df_visitantes['MATCHUP'] = df_visitantes['MATCHUP'].astype(str)

C:\Users\iauzm\AppData\Local\Temp\ipykernel_4152\2735282578.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_locales['MATCHUP'] = df_locales['MATCHUP'].astype(str)
C:\Users\iauzm\AppData\Local\Temp\ipykernel_4152\2735282578.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_visitantes['MATCHUP'] = df_visitantes['MATCHUP'].astype(str)


In [56]:
df_locales['MATCHUP'] = df_locales['MATCHUP'].apply(lambda x:x.replace('@','VS') )
df_visitantes['MATCHUP'] = df_visitantes['MATCHUP'].apply(lambda x:x.replace('@','VS'))
df_locales['MATCHUP'] = df_locales['MATCHUP'].apply(lambda x:x.replace('vs.','VS') )
df_visitantes['MATCHUP'] = df_visitantes['MATCHUP'].apply(lambda x:x.replace('vs.','VS'))

C:\Users\iauzm\AppData\Local\Temp\ipykernel_4152\29426620.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_locales['MATCHUP'] = df_locales['MATCHUP'].apply(lambda x:x.replace('@','VS') )
C:\Users\iauzm\AppData\Local\Temp\ipykernel_4152\29426620.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_visitantes['MATCHUP'] = df_visitantes['MATCHUP'].apply(lambda x:x.replace('@','VS'))
C:\Users\iauzm\AppData\Local\Temp\ipykernel_4152\29426620.py:3: SettingWithCopyWarning: 
A value is trying to be set on a

In [57]:
df_visitantes.head()

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,...,PF,PLUS_MINUS,fecha,merge_column,equipo_1,equipo_2,Unnamed: 0,Date,Visitor/Neutral,Home/Neutral
0,22023.0,1.610613e+09,ATL,Atlanta Hawks,22300172.0,2023-11-09,ATL VS ORL,W,240.0,120.0,...,25.0,1.0,2023-011-09,2023-011-09Atlanta Hawks,ATL,ORL,NaN,NaN,NaN,NaN
1,22023.0,1.610613e+09,IND,Indiana Pacers,22300171.0,2023-11-09,IND VS MIL,W,239.0,126.0,...,25.0,2.0,2023-011-09,2023-011-09Indiana Pacers,IND,MIL,NaN,NaN,NaN,NaN
2,22023.0,1.610613e+09,MIL,Milwaukee Bucks,22300171.0,2023-11-09,MIL VS IND,L,240.0,124.0,...,19.0,-2.0,2023-011-09,2023-011-09Milwaukee Bucks,MIL,IND,NaN,NaN,NaN,NaN
3,22023.0,1.610613e+09,ORL,Orlando Magic,22300172.0,2023-11-09,ORL VS ATL,L,240.0,119.0,...,24.0,-1.0,2023-011-09,2023-011-09Orlando Magic,ORL,ATL,NaN,NaN,NaN,NaN
4,62023.0,1.610613e+09,IND,Indiana Pacers,62300001.0,2023-12-09,IND VS LAL,L,238.0,109.0,...,33.0,-14.0,2023-012-09,2023-012-09Indiana Pacers,IND,LAL,NaN,NaN,NaN,NaN


In [58]:
df_visitantes['merge_2']=df_visitantes['equipo_2']+' VS '+df_visitantes['equipo_1']

C:\Users\iauzm\AppData\Local\Temp\ipykernel_4152\1693181032.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_visitantes['merge_2']=df_visitantes['equipo_2']+' VS '+df_visitantes['equipo_1']


In [59]:
df_locales['merge_2']=df_locales['MATCHUP']

C:\Users\iauzm\AppData\Local\Temp\ipykernel_4152\130368488.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_locales['merge_2']=df_locales['MATCHUP']


In [60]:
prueba= df_locales.merge(df_visitantes,on='merge_2')

In [61]:
prueba.columns

Index(['SEASON_ID_x', 'TEAM_ID_x', 'TEAM_ABBREVIATION_x', 'TEAM_NAME_x',
       'GAME_ID_x', 'GAME_DATE_x', 'MATCHUP_x', 'WL_x', 'MIN_x', 'PTS_x',
       'FGM_x', 'FGA_x', 'FG_PCT_x', 'FG3M_x', 'FG3A_x', 'FG3_PCT_x', 'FTM_x',
       'FTA_x', 'FT_PCT_x', 'OREB_x', 'DREB_x', 'REB_x', 'AST_x', 'STL_x',
       'BLK_x', 'TOV_x', 'PF_x', 'PLUS_MINUS_x', 'fecha_x', 'merge_column_x',
       'equipo_1_x', 'equipo_2_x', 'Unnamed: 0_x', 'Date_x',
       'Visitor/Neutral_x', 'Home/Neutral_x', 'merge_2', 'SEASON_ID_y',
       'TEAM_ID_y', 'TEAM_ABBREVIATION_y', 'TEAM_NAME_y', 'GAME_ID_y',
       'GAME_DATE_y', 'MATCHUP_y', 'WL_y', 'MIN_y', 'PTS_y', 'FGM_y', 'FGA_y',
       'FG_PCT_y', 'FG3M_y', 'FG3A_y', 'FG3_PCT_y', 'FTM_y', 'FTA_y',
       'FT_PCT_y', 'OREB_y', 'DREB_y', 'REB_y', 'AST_y', 'STL_y', 'BLK_y',
       'TOV_y', 'PF_y', 'PLUS_MINUS_y', 'fecha_y', 'merge_column_y',
       'equipo_1_y', 'equipo_2_y', 'Unnamed: 0_y', 'Date_y',
       'Visitor/Neutral_y', 'Home/Neutral_y'],
      dtype='obj

In [63]:

df_locales.head()

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,...,PLUS_MINUS,fecha,merge_column,equipo_1,equipo_2,Unnamed: 0,Date,Visitor/Neutral,Home/Neutral,merge_2
6,22023.0,1.610613e+09,DEN,Denver Nuggets,22300061.0,2023-10-24,DEN VS LAL,W,240.0,119.0,...,12.0,2023-10-24,2023-10-24Denver Nuggets,DEN,LAL,0.0,2023-10-24,Los Angeles Lakers,Denver Nuggets,DEN VS LAL
7,22023.0,1.610613e+09,GSW,Golden State Warriors,22300062.0,2023-10-24,GSW VS PHX,L,240.0,104.0,...,-4.0,2023-10-24,2023-10-24Golden State Warriors,GSW,PHX,1.0,2023-10-24,Phoenix Suns,Golden State Warriors,GSW VS PHX
12,22023.0,1.610613e+09,BKN,Brooklyn Nets,22300067.0,2023-10-25,BKN VS CLE,L,240.0,113.0,...,-1.0,2023-10-25,2023-10-25Brooklyn Nets,BKN,CLE,8.0,2023-10-25,Cleveland Cavaliers,Brooklyn Nets,BKN VS CLE
13,22023.0,1.610613e+09,CHA,Charlotte Hornets,22300063.0,2023-10-25,CHA VS ATL,W,240.0,116.0,...,6.0,2023-10-25,2023-10-25Charlotte Hornets,CHA,ATL,5.0,2023-10-25,Atlanta Hawks,Charlotte Hornets,CHA VS ATL
14,22023.0,1.610613e+09,CHI,Chicago Bulls,22300070.0,2023-10-25,CHI VS OKC,L,242.0,104.0,...,-20.0,2023-10-25,2023-10-25Chicago Bulls,CHI,OKC,10.0,2023-10-25,Oklahoma City Thunder,Chicago Bulls,CHI VS OKC


In [64]:
df_visitantes.sort_values(by='GAME_DATE')

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,...,PLUS_MINUS,fecha,merge_column,equipo_1,equipo_2,Unnamed: 0,Date,Visitor/Neutral,Home/Neutral,merge_2
8,22023.0,1.610613e+09,LAL,Los Angeles Lakers,22300061.0,2023-10-24,LAL VS DEN,L,239.0,107.0,...,-12.0,2023-10-24,2023-10-24Los Angeles Lakers,LAL,DEN,NaN,NaN,NaN,NaN,DEN VS LAL
9,22023.0,1.610613e+09,PHX,Phoenix Suns,22300062.0,2023-10-24,PHX VS GSW,W,241.0,108.0,...,4.0,2023-10-24,2023-10-24Phoenix Suns,PHX,GSW,NaN,NaN,NaN,NaN,GSW VS PHX
25,22023.0,1.610613e+09,NOP,New Orleans Pelicans,22300071.0,2023-10-25,NOP VS MEM,W,240.0,111.0,...,7.0,2023-10-25,2023-10-25New Orleans Pelicans,NOP,MEM,NaN,NaN,NaN,NaN,MEM VS NOP
34,22023.0,1.610613e+09,WAS,Washington Wizards,22300064.0,2023-10-25,WAS VS IND,L,238.0,120.0,...,-23.0,2023-10-25,2023-10-25Washington Wizards,WAS,IND,NaN,NaN,NaN,NaN,IND VS WAS
30,22023.0,1.610613e+09,SAC,Sacramento Kings,22300072.0,2023-10-25,SAC VS UTA,W,239.0,130.0,...,16.0,2023-10-25,2023-10-25Sacramento Kings,SAC,UTA,NaN,NaN,NaN,NaN,UTA VS SAC
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2663,42023.0,1.610613e+09,DAL,Dallas Mavericks,42300312.0,2024-05-24,DAL VS MIN,W,240.0,109.0,...,1.0,2024-05-24,2024-05-24Dallas Mavericks,DAL,MIN,NaN,NaN,NaN,NaN,MIN VS DAL
2665,42023.0,1.610613e+09,BOS,Boston Celtics,42300303.0,2024-05-25,BOS VS IND,W,239.0,114.0,...,3.0,2024-05-25,2024-05-25Boston Celtics,BOS,IND,NaN,NaN,NaN,NaN,IND VS BOS
2668,42023.0,1.610613e+09,MIN,Minnesota Timberwolves,42300313.0,2024-05-26,MIN VS DAL,L,239.0,107.0,...,-9.0,2024-05-26,2024-05-26Minnesota Timberwolves,MIN,DAL,NaN,NaN,NaN,NaN,DAL VS MIN
2669,42023.0,1.610613e+09,BOS,Boston Celtics,42300304.0,2024-05-27,BOS VS IND,W,240.0,105.0,...,3.0,2024-05-27,2024-05-27Boston Celtics,BOS,IND,NaN,NaN,NaN,NaN,IND VS BOS


In [65]:
df_visitantes['merge_2']=df_visitantes['fecha']+df_visitantes['merge_2']
df_locales['merge_2']=df_locales['fecha']+df_locales['merge_2']

C:\Users\iauzm\AppData\Local\Temp\ipykernel_4152\1201860148.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_visitantes['merge_2']=df_visitantes['fecha']+df_visitantes['merge_2']
C:\Users\iauzm\AppData\Local\Temp\ipykernel_4152\1201860148.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_locales['merge_2']=df_locales['fecha']+df_locales['merge_2']


In [66]:
df_merged= df_locales.merge(df_visitantes,on='merge_2')

In [67]:
print(df_locales.shape)
print(df_visitantes.shape)
print(df_merged.shape)
df_merged

(1314, 37)
(1360, 37)
(1266, 73)


,SEASON_ID_x,TEAM_ID_x,TEAM_ABBREVIATION_x,TEAM_NAME_x,GAME_ID_x,GAME_DATE_x,MATCHUP_x,WL_x,MIN_x,PTS_x,...,PF_y,PLUS_MINUS_y,fecha_y,merge_column_y,equipo_1_y,equipo_2_y,Unnamed: 0_y,Date_y,Visitor/Neutral_y,Home/Neutral_y
0,22023.0,1.610613e+09,DEN,Denver Nuggets,22300061.0,2023-10-24,DEN VS LAL,W,240.0,119.0,...,18.0,-12.0,2023-10-24,2023-10-24Los Angeles Lakers,LAL,DEN,NaN,NaN,NaN,NaN
1,22023.0,1.610613e+09,GSW,Golden State Warriors,22300062.0,2023-10-24,GSW VS PHX,L,240.0,104.0,...,22.0,4.0,2023-10-24,2023-10-24Phoenix Suns,PHX,GSW,NaN,NaN,NaN,NaN
2,22023.0,1.610613e+09,BKN,Brooklyn Nets,22300067.0,2023-10-25,BKN VS CLE,L,240.0,113.0,...,23.0,1.0,2023-10-25,2023-10-25Cleveland Cavaliers,CLE,BKN,NaN,NaN,NaN,NaN
3,22023.0,1.610613e+09,CHA,Charlotte Hornets,22300063.0,2023-10-25,CHA VS ATL,W,240.0,116.0,...,19.0,-6.0,2023-10-25,2023-10-25Atlanta Hawks,ATL,CHA,NaN,NaN,NaN,NaN
4,22023.0,1.610613e+09,CHI,Chicago Bulls,22300070.0,2023-10-25,CHI VS OKC,L,242.0,104.0,...,19.0,20.0,2023-10-25,2023-10-25Oklahoma City Thunder,OKC,CHI,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1261,42023.0,1.610613e+09,MIN,Minnesota Timberwolves,42300312.0,2024-05-24,MIN VS DAL,L,239.0,108.0,...,17.0,1.0,2024-05-24,2024-05-24Dallas Mavericks,DAL,MIN,NaN,NaN,NaN,NaN
1262,42023.0,1.610613e+09,IND,Indiana Pacers,42300303.0,2024-05-25,IND VS BOS,L,239.0,111.0,...,18.0,3.0,2024-05-25,2024-05-25Boston Celtics,BOS,IND,NaN,NaN,NaN,NaN
1263,42023.0,1.610613e+09,DAL,Dallas Mavericks,42300313.0,2024-05-26,DAL VS MIN,W,239.0,116.0,...,24.0,-9.0,2024-05-26,2024-05-26Minnesota Timberwolves,MIN,DAL,NaN,NaN,NaN,NaN
1264,42023.0,1.610613e+09,IND,Indiana Pacers,42300304.0,2024-05-27,IND VS BOS,L,240.0,102.0,...,18.0,3.0,2024-05-27,2024-05-27Boston Celtics,BOS,IND,NaN,NaN,NaN,NaN


In [68]:
df_merged.to_csv('partidos_merged')

### Cambio el nombre de las columnas para verlo más claramente

In [72]:
df_merged_ren= df_merged.rename(columns={  'TEAM_ABBREVIATION_x':'abreviación_eq_local', 'TEAM_NAME_x':'equipo_local',
        'GAME_DATE_x':'fecha', 'MATCHUP_x':'cabecera', 'WL_x':'WL_local', 'PTS_x':'pts_local',
       'FGM_x':'FGM_local', 'FGA_x':'FGA_local', 'FG_PCT_x':'FG_PCT_local', 'FG3M_x':'FG3M_local', 'FG3A_x':'FG3A_local', 'FG3_PCT_x':'FG3_PCT_local', 'FTM_x':'FTM_local',
       'FTA_x':'FTA_local', 'FT_PCT_x':'FT_PCT_local', 'OREB_x':'OREB_local', 'DREB_x':'DREB_local', 'REB_x':'REB_local', 'AST_x':'AST_local', 'STL_x':'STL_local',
       'BLK_x':'BLK_local', 'TOV_x': 'TOV_local', 'PF_x':'PF_local', 'PLUS_MINUS_x':'PLUS_MINUS_local', 
 'TEAM_ABBREVIATION_y':'TEAM_ABBREVIATION_visitante', 'TEAM_NAME_y':'equipo_visitante',
       'WL_y':'WL_visitante', 'PTS_y':'pts_visitante', 'FGM_y':'FGM_visitante', 'FGA_y':'FGA_visitante',
       'FG_PCT_y':'FG_PCT_visitante', 'FG3M_y':'FG3M_visitante', 'FG3A_y':'FG3A_visitante', 'FG3_PCT_y':'FG3_PCT_visitante', 'FTM_y':'FTM_visitante', 'FTA_y':'FTA_visitante',
       'FT_PCT_y':'FT_PCT_visitante', 'OREB_y': 'OREB_visitante', 'DREB_y':'DREB_visitante', 'REB_y':'REB_visitante', 'AST_y':'AST_visitante', 'STL_y':'STL_visitante', 'BLK_y':'BLK_visitante',
       'TOV_y':'TOV_visitante', 'PF_y':'PF_visitante', 'PLUS_MINUS_y':'PLUS_MINUS_visitante'})

In [73]:
df_merged_ren =df_merged_ren[['abreviación_eq_local', 'equipo_local',
       'fecha', 'cabecera', 'WL_local','pts_local',
      'FGM_local', 'FGA_local', 'FG_PCT_local','FG3M_local','FG3A_local', 'FG3_PCT_local', 'FTM_local',
       'FTA_local', 'FT_PCT_local', 'OREB_local','DREB_local', 'REB_local', 'AST_local','STL_local',
       'BLK_local',  'TOV_local', 'PF_local', 'PLUS_MINUS_local', 
 'TEAM_ABBREVIATION_visitante', 'equipo_visitante',
       'WL_visitante', 'pts_visitante','FGM_visitante', 'FGA_visitante',
       'FG_PCT_visitante', 'FG3M_visitante','FG3A_visitante', 'FG3_PCT_visitante', 'FTM_visitante', 'FTA_visitante',
       'FT_PCT_visitante', 'OREB_visitante','DREB_visitante', 'REB_visitante', 'AST_visitante', 'STL_visitante', 'BLK_visitante',
       'TOV_visitante', 'PF_visitante', 'PLUS_MINUS_visitante'
       
]] 

In [74]:

df_merged_ren.to_csv('partidos_24_limpio')
